In [73]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings


warnings.filterwarnings("ignore")

df311 = pd.read_csv(r"C:\Users\rebel\UE_work\Urban_Echo\311_cleaned.csv")

df311.head()

,Address,City Council District,Department,Service Request Type,ERT (Estimated Response Time),Status,Priority,Unique Key,Lat_Long Location
0,"2701 CANTON ST, DALLAS, TX, 75226",2,Code Compliance,Code Concern - CCS,4 Business Days,Closed,Standard,Standard,"(32.78202068982446000,-96.78441935829460000)"
1,"734 RAYENELL AVE, DALLAS, TX, 75217",5,Water Utilities,Storm Sewer Line Locate - DWU,5 Business Days,Closed (Duplicate),Standard,Standard,"(32.72536547086228000,-96.70604084210902000)"
2,"933 FERNWOOD AVE, DALLAS, TX, 75216",4,Code Compliance,Code Concern - CCS,4 Business Days,Closed,Standard,Standard,"(32.73791239975091000,-96.80930802963174000)"
3,"6216 MOONGLOW DR, DALLAS, TX, 75241",3,Code Compliance,Code Concern - CCS,4 Business Days,Closed,Standard,Standard,"(32.67258130750240000,-96.79714596689998000)"
4,"1620 TAMPA BAY DR, DALLAS, TX, 75217",8,Code Compliance,Code Concern - CCS,4 Business Days,Closed,Standard,Standard,"(32.68602419361726400,-96.66254883934802000)"


In [74]:
# df311 = df311[df311["City Council District"].str.contains(",") == False]
df311['City Council District'] = df311['City Council District'].astype('category')
df311['Address'] = df311['Address'].astype('category')
df311['Department'] = df311['Department'].astype('category')
df311['Service Request Type'] = df311['Service Request Type'].astype('category')
df311['ERT (Estimated Response Time)'] = df311['ERT (Estimated Response Time)'].astype('category')
df311['Status'] = df311['Status'].astype('category')
df311['Priority'] = df311['Priority'].astype('category')
# df311['Unique Key'] = df311['Priority'].astype('category')
# df311['Lat_Long Location'] = df311['Lat_Long Location'].astype('category')
df311 = df311.drop(columns='Lat_Long Location')
df311 = df311.drop(columns= 'Unique Key')
df311 = df311.drop(columns="Address")
print(df311.dtypes)

City Council District            category
Department                       category
Service Request Type             category
ERT (Estimated Response Time)    category
Status                           category
Priority                         category
dtype: object


In [75]:
df311.describe(exclude=np.number)

,City Council District,Department,Service Request Type,ERT (Estimated Response Time),Status,Priority
count,1395463,1395463,1395463,1395463,1395463,1395463
unique,14,29,247,18,14,7
top,4,Code Compliance,Code Concern - CCS,4 Business Days,Closed,Standard
freq,148954,401131,375306,378579,1289645,1031033


In [76]:
# convert the dtypes from object type to float, int64, etc.
# There are string values contained within this column - change the 
# 311_cleaned.csv file
#df311["City Council District"].astype(np.int64)

In [77]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = df311.drop('City Council District', axis=1), df311[['City Council District']]

In [78]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
#print(X_train.dtypes)

In [79]:
# Assuming df311['City Council District'] contains your categorical labels
# Instantiate LabelEncoder

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit label encoder and transform the labels
encoded_labels = label_encoder.fit_transform(df311['City Council District'])

dtrain_clf = xgb.DMatrix(data=X_train, label=encoded_labels[:len(X_train)], enable_categorical=True)
dtest_clf = xgb.DMatrix(data=X_test, label=encoded_labels[len(X_train):], enable_categorical=True)


In [80]:
# import xgboost as xgb

# # Create regression matrices
# dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [81]:
import numpy as np 

In [82]:
# # Define hyperparameters
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 100
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
# )

In [83]:
# #evaluation
# # from sklearn.metrics import mean_squared_errors

# preds = model.predict(dtest_reg)

In [84]:
# rmse = mean_squared_error(y_test, preds, squared=False)

# print(f"RMSE of the base model: {rmse:.3f}")

In [85]:
# evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
#    evals=evals,
#    verbose_eval=500,
#    early_stopping_rounds=50
# )

In [86]:
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 500

# results = xgb.cv(
#    params, dtrain_reg,
#    num_boost_round=n,
#    nfold=5,
#    early_stopping_rounds=20
# )

# # This gives us the results of the spilts
# results.head()

# # We want to find the minimum rmse-mean 
# best_rmse = results['test-rmse-mean'].min()

# best_rmse



In [87]:
# len(np.unique(df311['City Council District']))

In [88]:
columns = df311.columns
for col in columns: 
    print(f"col = {col} {len(np.unique(df311[col]))}")

col = City Council District 14
col = Department 29
col = Service Request Type 247
col = ERT (Estimated Response Time) 18
col = Status 14
col = Priority 7


In [89]:
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": len(np.unique(df311['City Council District']))}
n = 1000

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=100,
   metrics=["mlogloss", "auc", "merror"],
)


In [90]:
results.keys()

Index(['train-mlogloss-mean', 'train-mlogloss-std', 'train-auc-mean',
       'train-auc-std', 'train-merror-mean', 'train-merror-std',
       'test-mlogloss-mean', 'test-mlogloss-std', 'test-auc-mean',
       'test-auc-std', 'test-merror-mean', 'test-merror-std'],
      dtype='object')

In [93]:
print(results['test-merror-mean'].max())

0.8940050468431096


In [94]:
# print("Results: ", results['test-merror-mean'].max())

for i in results.keys():
    print(f"Results = {i} {results[i].max()}")
    #print("Results: ", results[i].max())

Results = train-mlogloss-mean 2.6284056731228898
Results = train-mlogloss-std 5.681175833791123e-05
Results = train-auc-mean 0.5144880896016023
Results = train-auc-std 0.0002522274784430746
Results = train-merror-mean 0.8925123519384759
Results = train-merror-std 0.00012066892688503666
Results = test-mlogloss-mean 2.629126280954563
Results = test-mlogloss-std 0.0002549977235506558
Results = test-auc-mean 0.4994789259593803
Results = test-auc-std 0.0006677674658970107
Results = test-merror-mean 0.8940050468431096
Results = test-merror-std 0.00037133817457341456
